In [2]:
import os
import json
import earthpy as et
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
%matplotlib inline

# change the global options that Geopandas inherits from
pd.set_option('display.max_columns',None)

In [51]:
# Change directory to landslide-detect data path
data_path = os.path.join(et.io.HOME, "earth-analytics", "landslide-detect")
if os.path.exists(data_path):
    os.chdir(data_path)
else:
    os.makedirs(data_path)
    print('The new directory is created!')
    os.chdir(data_path)

print('Current working directory is set to: ', os.getcwd())

Current working directory is set to:  C:\Users\mthompson1\earth-analytics\landslide-detect


In [54]:
def create_dataframe(data_file):
    """
    Reads a csv file into a Pandas DataFrame. Sets the index to the 'slide.id'
    and sorts the DataFrame by the new index. Creates a new column 'point' by 
    merging two other columns.

    Parameters
    ----------
    data_file : str

    Returns
    -------
    dataframe
        sorted by index
    """

    df = pd.read_csv(data_file, index_col='slide.id').sort_values(by='slide.id')
    df['point'] = df['lon'].map(str) + ', ' + df['lat'].map(str)

    return df

landslide_df = create_dataframe('landslides.verified.csv')
landslide_df.head()

,slide.date,location,type,trigger,size,lon,lat,location_accuracy,event_title,admin_division_name,ge.lat,ge.lon,is.exact,slide.index,point
slide.id,,,,,,,,,,,,,,,
7392,2015-12-13T19:45:00Z,Interstate 5 in the Fort Tejon area,mudslide,downpour,medium,-118.896200,34.881300,10km,Interstate 5 in the Fort Tejon area,California,NaN,NaN,False,17,"-118.8962, 34.8813"
7534,2015-11-29T00:00:00Z,Near the twin bridges on the Tryon side of 176,rock_fall,rain,medium,-82.321600,35.220700,5km,Near the twin bridges on the Tryon side of 176,North Carolina,NaN,NaN,False,48,"-82.3216, 35.2207"
7536,2016-02-16T15:00:00Z,Tyee Access Road,rock_fall,rain,medium,-123.568400,43.433000,1km,Tyee Access Road,Oregon,43.433000,-123.568400,True,69,"-123.5684, 43.433"
7538,2016-01-07T05:55:00Z,Highway 140 between El Portal and Yosemite Va...,rock_fall,rain,medium,-119.724608,37.698564,25km,Highway 140 between El Portal and Yosemite Va...,California,37.698564,-119.724608,True,64,"-119.72460833333334, 37.69856388888888"
7555,2016-03-11T00:00:00Z,Route 1 north of Westport,landslide,rain,medium,-123.797464,39.694056,exact,Route 1 north of Westport,California,39.694056,-123.797464,True,66,"-123.79746388888888, 39.69405555555555"


In [55]:
# for id, location, point in zip(landslide_df.index,
#                                landslide_df['location'],
#                                landslide_df['point'],
#                                ):
#     print(id, location, point)


In [ ]:
def image_search(point, start, stop):
    """
    Iterates through a pandas DataFrame or csv and searches Google Earth
    Engine based on center AOI point, start time, and stop time.
    
    
    """

    # point + some padded boundary
    # start = slide.date - some time in days
    # stop = start + some time in days

    # loop over dataframe to search on each point, start, and stop
    
        im_coll = (ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT')
                .filterBounds(point)
                .filterDate(ee.Date('start'), ee.Date('stop'))
                .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation',
                                                'VV'))
                .filter(ee.Filter.listContains('transmitterReceiverPolarisation',
                                                'VH'))
                .map(lambda img: img.set('date',
                                                ee.Date(img.date()).format('YYYYMMdd')))
                .sort('date'))

        timestamplist = (im_coll.aggregate_array('date')
                        .map(lambda d: ee.String('T').cat(ee.String(d)))
                        .getInfo())
    return timestamplist